#### Install the required packages

In [1]:
# %conda install pandas geopandas matplotlib
# %conda install -c conda-forge folium pyarrow

#### Write GeoParquet using GeoPandas

In [2]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib as plt

In [3]:
# Read in the county shapefile and census data csv
counties = gpd.read_file(r"tl_2021_us_county/tl_2021_us_county.shp")

# Read in county census table from here https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_70769902
cendat = pd.read_csv("co-est2021-alldata.csv",  encoding='ISO-8859-1')

In [4]:
# Sort by county name and show the counties GeoDataframe
counties = counties.sort_values(by="NAMELSAD")
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
2210,45,001,01245666,45001,Abbeville,Abbeville County,06,H1,G4020,None,None,None,A,1272184102,53221051,+34.2290410,-082.4540577,"POLYGON ((-82.50085 34.07190, -82.50093 34.071..."
3045,22,001,00558389,22001,Acadia,Acadia Parish,15,H1,G4020,318,29180,None,A,1697073813,5934788,+30.2914970,-092.4110366,"POLYGON ((-92.36150 30.07855, -92.36294 30.077..."
1987,51,001,01480091,51001,Accomack,Accomack County,06,H1,G4020,None,None,None,A,1163725929,2229259960,+37.7659435,-075.7578073,"POLYGON ((-75.51693 37.65483, -75.51720 37.653..."
1679,16,001,00395066,16001,Ada,Ada County,06,H1,G4020,147,14260,None,A,2724705225,22159206,+43.4514767,-116.2443760,"POLYGON ((-115.98028 43.58595, -115.98018 43.5..."
1947,29,001,00765805,29001,Adair,Adair County,06,H1,G4020,None,28860,None,A,1469362059,5468507,+40.1906655,-092.6035922,"POLYGON ((-92.34862 40.21583, -92.34784 40.215..."


In [5]:
# Sort by county name and show the county census Dataframe
cendat = cendat.sort_values(by="CTYNAME")
cendat.head()

# We can see that the county name fields are the same - we will join the census data to the counties based on county name

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,...,RESIDUAL2021,GQESTIMATESBASE2020,GQESTIMATES2020,GQESTIMATES2021,RBIRTH2021,RDEATH2021,RNATURALCHG2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021
2358,50,3,5,45,1,South Carolina,Abbeville County,24295,24212,24299,...,1,807,807,807,9.564841,13.357795,-3.792954,0.000000,7.338542,7.338542
1133,50,3,7,22,1,Louisiana,Acadia Parish,57576,57455,57288,...,11,759,759,759,14.292811,14.519404,-0.226593,0.034861,-2.910853,-2.875992
2868,50,3,5,51,1,Virginia,Accomack County,33413,33348,33246,...,3,407,407,407,9.550410,16.818332,-7.267922,0.961048,3.153437,4.114485
565,50,4,8,16,1,Idaho,Ada County,494967,497984,511931,...,-126,11167,11167,11167,9.533476,8.188808,1.344668,0.100999,26.424006,26.525005
1510,50,2,4,29,1,Missouri,Adair County,25314,25246,25185,...,3,2827,2827,2827,11.024965,9.002399,2.022565,1.824275,-6.384962,-4.560687


In [6]:
# Join the county GeoDataFrame and census Dataframe
county_cendat = counties.merge(cendat, left_on="NAMELSAD", right_on='CTYNAME')
county_cendat.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,RESIDUAL2021,GQESTIMATESBASE2020,GQESTIMATES2020,GQESTIMATES2021,RBIRTH2021,RDEATH2021,RNATURALCHG2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021
0,45,001,01245666,45001,Abbeville,Abbeville County,06,H1,G4020,None,...,1,807,807,807,9.564841,13.357795,-3.792954,0.000000,7.338542,7.338542
1,22,001,00558389,22001,Acadia,Acadia Parish,15,H1,G4020,318,...,11,759,759,759,14.292811,14.519404,-0.226593,0.034861,-2.910853,-2.875992
2,51,001,01480091,51001,Accomack,Accomack County,06,H1,G4020,None,...,3,407,407,407,9.550410,16.818332,-7.267922,0.961048,3.153437,4.114485
3,16,001,00395066,16001,Ada,Ada County,06,H1,G4020,147,...,-126,11167,11167,11167,9.533476,8.188808,1.344668,0.100999,26.424006,26.525005
4,29,001,00765805,29001,Adair,Adair County,06,H1,G4020,None,...,3,2827,2827,2827,11.024965,9.002399,2.022565,1.824275,-6.384962,-4.560687


In [8]:
# new_gdf = pd.DataFrame(county_cendat.loc[county_cendat["POPESTIMATE2021"]>511000, "NAMELSAD"])

# new_gdf = county_cendat[county_cendat["POPESTIMATE2021"] > 500000]
new_gdf = county_cendat[["NAMELSAD", "geometry"]]

In [9]:
new_gdf

,NAMELSAD,geometry
0,Abbeville County,"POLYGON ((-82.50085 34.07190, -82.50093 34.071..."
1,Acadia Parish,"POLYGON ((-92.36150 30.07855, -92.36294 30.077..."
2,Accomack County,"POLYGON ((-75.51693 37.65483, -75.51720 37.653..."
3,Ada County,"POLYGON ((-115.98028 43.58595, -115.98018 43.5..."
4,Adair County,"POLYGON ((-92.34862 40.21583, -92.34784 40.215..."
...,...,...
14531,Yuma County,"POLYGON ((-114.47325 33.02788, -114.45989 33.0..."
14532,Yuma County,"POLYGON ((-114.47325 33.02788, -114.45989 33.0..."
14533,Zapata County,"POLYGON ((-99.15217 27.27142, -99.15088 27.271..."
14534,Zavala County,"POLYGON ((-99.41160 28.81923, -99.41159 28.818..."


In [10]:
# Write GeoDataFrame to GeoParquet
new_gdf.to_parquet("us_county_pop.parquet")

/tmp/ipykernel_4927/3901980015.py:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  new_gdf.to_parquet("us_county_pop.parquet")


: 

: 

#### Read GeoParquet and display

In [ ]:
gdf = gpd.read_parquet("us_county_pop.parquet")

In [ ]:
# Convert WKT geometry column to json
geo = (gpd.GeoSeries(gdf.set_index('name')['geometry'])).to_json()

In [ ]:
# Create Folium interactive map and plot the data

map = folium.Map(location=[20.0,0.0], zoom_start=2, tiles=None)
folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(map)

scale = (gdf['gdp_md_est'].quantile((0,0.1,0.75,0.9,1))).tolist()

folium.Choropleth(
 geo_data=geo,
 name='GeoParquet Choropleth',
 data=gdf,
 columns=['name','gdp_md_est'],
 key_on="feature.id",
 fill_color='YlGnBu',
 threshold_scale=scale,
 fill_opacity=1,
 line_opacity=0.2,
 legend_name='Estimated GDP (millions $)',
 smooth_factor=0
).add_to(map)
map